Import the libraries needed

In [1]:
import pandas as pd
import numpy as np
import os
import pyodbc
pyodbc.drivers()
from datetime import datetime
import psutil

# For the Yahoo Finance api
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

# importing matplotlib module
import matplotlib.pyplot as plt
plt.style.use('default')

C:\Users\m033\Anaconda3\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (3.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(


In [2]:
pd.set_option('max_row', None)
#pd.options.display.max_rows = 1000

# Set a value for the home folder.
home_folder = "."

# Set values for the various paths.
input_path = home_folder + "\data"

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

2024-08-14 17:43:32.697832
svmem(total=16756752384, available=4995297280, percent=70.2, used=11761455104, free=4995297280)


### All the data used has been provisined in Get_The_Data.ipynb

# Investigated the spike and it is downw to a fund closure on those dates.

In [30]:
![Capture.PNG](attachment:Capture.PNG)

'[Capture.PNG]' is not recognized as an internal or external command,
operable program or batch file.


# 1. Read back the dataset of all switches.

In [3]:
# giving directory name
All_Switches = input_path + '\All_Switches.csv'

df_All_Switches = pd.read_csv(All_Switches, delimiter=',',low_memory=False, header=0)

# Filter out old dates.
df_All_Switches = df_All_Switches[df_All_Switches.PROCESSING_DATE > '20181231']

# Make the processing date a real date.
df_All_Switches['PROCESSING_DATE'] = pd.to_datetime(df_All_Switches['PROCESSING_DATE'])

# print stats.
print(len(df_All_Switches.index))
print(df_All_Switches.dtypes)

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

# No displays to protect PII data
#df_All_Switches.head(5)

202891
index                              int64
POLICY_NUMBER                     object
LEGACY_FUND_CODE                  object
PRODUCT                           object
PROCESSING_DATE           datetime64[ns]
EFFECTIVE_DATE                    object
UNIT_TYPE                         object
CHARGE_TYPE                       object
CASH_VALUE                       float64
RISK_COMMENCEMENT_DATE            object
Servicing_Broker_Code             object
Issue_Date                        object
Cover_End_Date                    object
Anniversary_Date                  object
Source                            object
SOURCE                            object
dtype: object
2024-08-14 17:43:33.997121
svmem(total=16756752384, available=4947120128, percent=70.5, used=11809632256, free=4947120128)


# 2. Create a new dataframe with switches for the fund close date range.

In [4]:
from_dt = '2022-04-01'
to_dt   = '2022-07-31'
df_Fund_Close_Period = df_All_Switches[(df_All_Switches['PROCESSING_DATE'] > from_dt) & (df_All_Switches['PROCESSING_DATE'] < to_dt)]

print(len(df_Fund_Close_Period.index))

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

# No displays to protect PII data
#df_Fund_Close_Period.head(5)

9893
2024-08-14 17:43:34.146892
svmem(total=16756752384, available=4945907712, percent=70.5, used=11810844672, free=4945907712)


# 3. Get a list of policies that were involved in a fund closure.

In [31]:
# These are the funds that were closed.
close_funds = ['XSFFF6']

#print('These are the funds that were closed.',close_funds)

# Find the rows where this exists
df_Fund_Close_Pols = df_Fund_Close_Period[df_Fund_Close_Period.LEGACY_FUND_CODE.isin(close_funds)]


# Find the rows where this exists
#df_Fund_Close_Pols = df_Fund_Close_Period[~df_Fund_Close_Period.LEGACY_FUND_CODE.isin(close_funds)]
df_Fund_Close_Pols = df_Fund_Close_Period[df_Fund_Close_Period.LEGACY_FUND_CODE.isin(close_funds)]

# Isolate teh policy number
df_Fund_Close_Unique_Pols = df_Fund_Close_Pols[['POLICY_NUMBER']]

# Drop duplicate policy numbers
df_Fund_Close_Unique_Pols = df_Fund_Close_Unique_Pols.drop_duplicates(subset=['POLICY_NUMBER'], keep='first')

df_Fund_Close_Unique_Pols = df_Fund_Close_Unique_Pols.rename(columns={"POLICY_NUMBER":"Pols_To_Delete"})
df_Fund_Close_Unique_Pols.head(5)

# Write out to a comma separated values file.
filename = input_path + '\Fund_Close_Unique_Pols.csv'
df_Fund_Close_Unique_Pols.to_csv(filename, encoding='utf-8', index=False)  

print(len(df_Fund_Close_Unique_Pols.index))
print(filename)

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

# No displays to protect PII data
#df_Fund_Close_Unique_Pols.head(5)

69
.\data\Fund_Close_Unique_Pols.csv
2024-08-14 17:45:16.943810
svmem(total=16756752384, available=5398556672, percent=67.8, used=11358195712, free=5398556672)


# 4. Using the policy numbers from the list of policies that were involved in a fund closure, create a list of switches to delete.

In [6]:
print(len(df_Fund_Close_Period.index))
print(len(df_Fund_Close_Unique_Pols.index))


9893
69


In [7]:
df_LEFT_A = pd.merge(df_Fund_Close_Period, df_Fund_Close_Unique_Pols, left_on = ['POLICY_NUMBER'], right_on = ['Pols_To_Delete'] , how = 'left') 
print(len(df_LEFT_A.index))

# Write out to a comma separated values file.
filename = input_path + '\df_LEFT_A.csv'
df_LEFT_A.to_csv(filename, encoding='utf-8', index=False)  

print(filename)

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

# No displays to protect PII data
#df_LEFT_A.head(25)

9893
.\data\df_LEFT_A.csv
2024-08-14 17:43:36.485844
svmem(total=16756752384, available=4926275584, percent=70.6, used=11830476800, free=4926275584)


In [8]:
print(len(df_Fund_Close_Period.index))
print(len(df_Fund_Close_Unique_Pols.index))
print(len(df_LEFT_A.index))

9893
69
9893


# 5. Delete the fund closure switches.

In [9]:
# nOW DELETE ALL ROWS WHERE Pols_To_Delete is not NULL
print(len(df_LEFT_A.index))

9893


In [10]:
#df = df_LEFT_A[df_LEFT_A.Pols_To_Delete != 'NaN']
df_LEFT_A = df_LEFT_A[df_LEFT_A.Pols_To_Delete.isnull()]
print(len(df_LEFT_A.index))


6978


In [11]:
df_Closed_Funds_Policies_Removed = df_LEFT_A.drop(['index','Pols_To_Delete'],axis=1)
print(len(df_Closed_Funds_Policies_Removed.index))
print(len(df_LEFT_A.index))

6978
6978


In [12]:
#df = df_LEFT_A[df_LEFT_A.Pols_To_Delete != 'NaN']
#df_Closed_Funds_Policies_Removed = df_LEFT_A[df_LEFT_A.Pols_To_Delete.isnull()]
print(len(df_Closed_Funds_Policies_Removed.index))
print(len(df_LEFT_A.index))

6978
6978


In [13]:
# Write out to a comma separated values file.
filename = input_path + '\Closed_Funds_Policies_Removed.csv'
df_Closed_Funds_Policies_Removed.to_csv(filename, encoding='utf-8', index=False)  

print(len(df_Closed_Funds_Policies_Removed.index))
print(len(df_Fund_Close_Period.index))

print(filename)

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

# No displays to protect PII data
#df_Closed_Funds_Policies_Removed.head(25)

6978
9893
.\data\Closed_Funds_Policies_Removed.csv
2024-08-14 17:43:39.394051
svmem(total=16756752384, available=4960862208, percent=70.4, used=11795890176, free=4960862208)


# 6. Create a dataframe with all other dates, outside teh fund closure period.

In [14]:
# Create a dataframe with all other dates.
from_dt = '2022-04-01'
to_dt = '2022-07-31'
df_Fund_NON_Close_Period = df_All_Switches[(df_All_Switches['PROCESSING_DATE'] < from_dt) | (df_All_Switches['PROCESSING_DATE'] > to_dt)]


# Write out to a comma separated values file.
filename = input_path + '\Fund_NON_Close_Period.csv'
df_Fund_NON_Close_Period.to_csv(filename, encoding='utf-8', index=False)  

print(len(df_Fund_NON_Close_Period.index))

print(filename)

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

# No displays to protect PII data
#df_Fund_NON_Close_Period.head(5)

192949
.\data\Fund_NON_Close_Period.csv
2024-08-14 17:43:43.334450
svmem(total=16756752384, available=4912447488, percent=70.7, used=11844304896, free=4912447488)


# 7. Merge them back to get a dataframe that has the full set of switches, MINUS switches for policies involved in the fund closure.

In [15]:
# Merge the dataframes.
df_All_Switches_Without_Fund_Close= df_Closed_Funds_Policies_Removed.append(df_Fund_NON_Close_Period)
#df_Switches_Without_Fund_Close = df_Switches_Without_Fund_Close.drop(['Pols_To_Delete','level_0','index'],axis=1)


print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

# No displays to protect PII data
#df_All_Switches_Without_Fund_Close.head(5)

2024-08-14 17:43:43.414002
svmem(total=16756752384, available=4882300928, percent=70.9, used=11874451456, free=4882300928)


In [16]:
print("df_Fund_Close_Period             :",len(df_Fund_Close_Period.index))
print('df_Closed_Funds_Policies_Removed :',len(df_Closed_Funds_Policies_Removed.index))
print('df_Fund_NON_Close_Period         :',len(df_Fund_NON_Close_Period.index))
print('df_All_Switches                  :',len(df_All_Switches.index))


df_Fund_Close_Period             : 9893
df_Closed_Funds_Policies_Removed : 6978
df_Fund_NON_Close_Period         : 192949
df_All_Switches                  : 202891


In [17]:
df_All_Switches_Without_Fund_Close.dtypes

POLICY_NUMBER                     object
LEGACY_FUND_CODE                  object
PRODUCT                           object
PROCESSING_DATE           datetime64[ns]
EFFECTIVE_DATE                    object
UNIT_TYPE                         object
CHARGE_TYPE                       object
CASH_VALUE                       float64
RISK_COMMENCEMENT_DATE            object
Servicing_Broker_Code             object
Issue_Date                        object
Cover_End_Date                    object
Anniversary_Date                  object
Source                            object
SOURCE                            object
index                            float64
dtype: object

# 8. Re-sort by policy number and processing date and write out file.

In [18]:
# Soryt by policy number and processing date.
#df_All_Switches_Without_Fund_Close = df_All_Switches_Without_Fund_Close.sort_values(['PROCESSING_DATE','POLICY_NUMBER','SOURCE']).reset_index(drop=False)
df_All_Switches_Without_Fund_Close = df_All_Switches_Without_Fund_Close.sort_values(['PROCESSING_DATE','POLICY_NUMBER','SOURCE'])

print(len(df_All_Switches_Without_Fund_Close.index))

# Write out to a comma separated values file.
filename = input_path + '\All_Switches_Without_Fund_Close.csv'
df_All_Switches_Without_Fund_Close.to_csv(filename, encoding='utf-8', index=False)      

print(filename)

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

# No displays to protect PII data
#df_All_Switches_Without_Fund_Close.head(10)

199927


C:\Users\m033\Anaconda3\lib\site-packages\pandas\core\internals\blocks.py:2106: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


.\data\All_Switches_Without_Fund_Close.csv
2024-08-14 17:43:47.123725
svmem(total=16756752384, available=4833099776, percent=71.2, used=11923652608, free=4833099776)


# 9. Get the switchable funds/switch counts.

In [19]:
filename = input_path + '\Full_Population_Count.csv'
df_Full_Population_Count = pd.read_csv(filename, delimiter=',',low_memory=False, header=0)
print(len(df_Full_Population_Count.index))

print(filename)

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

# No displays to protect PII data
#df_Full_Population_Count.head(5)

66
.\data\Full_Population_Count.csv
2024-08-14 17:43:47.157793
svmem(total=16756752384, available=4827107328, percent=71.2, used=11929645056, free=4827107328)


# 10. Create the Fund Switch Count dataframe.

In [20]:
df_All_Switches_Without_Fund_Close.dtypes

POLICY_NUMBER                     object
LEGACY_FUND_CODE                  object
PRODUCT                           object
PROCESSING_DATE           datetime64[ns]
EFFECTIVE_DATE                    object
UNIT_TYPE                         object
CHARGE_TYPE                       object
CASH_VALUE                       float64
RISK_COMMENCEMENT_DATE            object
Servicing_Broker_Code             object
Issue_Date                        object
Cover_End_Date                    object
Anniversary_Date                  object
Source                            object
SOURCE                            object
index                            float64
dtype: object

In [21]:
df_Switches_Count = df_All_Switches_Without_Fund_Close[['POLICY_NUMBER','PROCESSING_DATE']]
#df_All_Switches['PROCESSING_DATE'] = pd.to_datetime(df_All_Switches['PROCESSING_DATE'].str.strip(), format='%Y/%m/%d')
df_Switches_Count['ProcessingtDt'] = df_Switches_Count['PROCESSING_DATE'].dt.strftime('%Y%m')

# Drop obsolete columns.
#df_Switches_Count = df_All_Switches.drop(['LEGACY_FUND_CODE','PRODUCT','EFFECTIVE_DATE','UNIT_TYPE',
#                                          'CHARGE_TYPE','CASH_VALUE','RISK_COMMENCEMENT_DATE','Cover_End_Date',
#                                         'index','Servicing_Broker_Code','Issue_Date','Anniversary_Date','SOURCE',
#                                          'PROCESSING_DATE','level_0','Pols_To_Delete'],axis=1)

# Stats
print(len(df_Switches_Count.index))

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

# No displays to protect PII data
#df_Switches_Count.head(5)

199927
2024-08-14 17:43:49.149839
svmem(total=16756752384, available=4837449728, percent=71.1, used=11919302656, free=4837449728)


C:\Users\m033\AppData\Local\Temp\ipykernel_24988\315763259.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Switches_Count['ProcessingtDt'] = df_Switches_Count['PROCESSING_DATE'].dt.strftime('%Y%m')


# 11. Drop duplicate policy numbers

In [22]:
# Drop duplicate policy numbers
df_Switches_Count = df_Switches_Count.drop_duplicates(subset=['POLICY_NUMBER','ProcessingtDt'], keep='first')
df_Switches_Count = df_Switches_Count.drop(['POLICY_NUMBER'],axis=1)

# print stats.
print(len(df_Switches_Count.index))

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

# No displays to protect PII data
#df_Switches_Count.head(5)

7811
2024-08-14 17:43:49.280023
svmem(total=16756752384, available=4829941760, percent=71.2, used=11926810624, free=4829941760)


In [23]:
# Reset teh index on count
df_Switches_Count=df_Switches_Count[['ProcessingtDt']].value_counts().reset_index(name='counts')

# Sort by ProcessingtDt
df_Switches_Count = df_Switches_Count.sort_values(by=['ProcessingtDt'])

print(len(df_Switches_Count.index))

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

# No displays to protect PII data
#df_Switches_Count.head(5)

68
2024-08-14 17:43:49.312001
svmem(total=16756752384, available=4834721792, percent=71.1, used=11922030592, free=4834721792)


In [24]:
df_All_Switches_Without_Fund_Close.dtypes

POLICY_NUMBER                     object
LEGACY_FUND_CODE                  object
PRODUCT                           object
PROCESSING_DATE           datetime64[ns]
EFFECTIVE_DATE                    object
UNIT_TYPE                         object
CHARGE_TYPE                       object
CASH_VALUE                       float64
RISK_COMMENCEMENT_DATE            object
Servicing_Broker_Code             object
Issue_Date                        object
Cover_End_Date                    object
Anniversary_Date                  object
Source                            object
SOURCE                            object
index                            float64
dtype: object

## Create the Fund Switch Count dataframe.

In [25]:
df_All_Switches = df_All_Switches_Without_Fund_Close
#df_All_Switches['PROCESSING_DATE'] = pd.to_datetime(df_All_Switches['PROCESSING_DATE'].str.strip(), format='%Y/%m/%d')
df_All_Switches['ProcessingtDt'] = df_All_Switches['PROCESSING_DATE'].dt.strftime('%Y%m')

# Drop obsolete columns.
df_Switches_Count = df_All_Switches.drop(['POLICY_NUMBER','LEGACY_FUND_CODE','PRODUCT','EFFECTIVE_DATE','UNIT_TYPE',
                                          'CHARGE_TYPE','CASH_VALUE','RISK_COMMENCEMENT_DATE','Cover_End_Date',
                                         'index','Servicing_Broker_Code','Issue_Date','Anniversary_Date','SOURCE','PROCESSING_DATE'],axis=1)

# Reset teh index on count
df_Switches_Count=df_Switches_Count[['ProcessingtDt']].value_counts().reset_index(name='counts')

# Sort by ProcessingtDt
df_Switches_Count = df_Switches_Count.sort_values(by=['ProcessingtDt'])

print(len(df_Switches_Count.index))

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

# No displays to protect PII data
#df_Switches_Count.head(5)

68
2024-08-14 17:43:51.755722
svmem(total=16756752384, available=4893585408, percent=70.8, used=11863166976, free=4893585408)


## Now remove all instances of these policies in the close fund period.

In [32]:
print(len(df_Fund_Close_Period.index))
#print(df_Fund_Close_Period.head(5))
#print(df_Fund_Close_Period.tail(5))


# Write out to a comma separated values file.
filename = input_path + '\Fund_Close_Period.csv'
#df_Fund_Close_Period.to_csv(filename, encoding='utf-8', index=False)  

print(filename)

9893
.\data\Fund_Close_Period.csv


In [27]:
print(len(df_Fund_Close_Unique_Pols.index))

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

# No displays to protect PII data
#df_Fund_Close_Unique_Pols.head(5)

69
2024-08-14 17:43:51.816905
svmem(total=16756752384, available=4884467712, percent=70.9, used=11872284672, free=4884467712)


## Remove close fund pols from the full list from that period.

In [28]:
close_fund_pols = df_Fund_Close_Unique_Pols.values.tolist()

# Stata

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

# No displays to protect PII data
#close_fund_pols

2024-08-14 17:43:51.833971
svmem(total=16756752384, available=4885299200, percent=70.8, used=11871453184, free=4885299200)


In [34]:
df_Fund_Close_Period_New = df_Fund_Close_Period[df_Fund_Close_Period.POLICY_NUMBER.isin(close_fund_pols)]

print(len(df_Fund_Close_Period_New.index))

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

# No displays to protect PII data
#df_Fund_Close_Period_New.head(5)

2024-08-14 17:46:40.363535
svmem(total=16756752384, available=5382082560, percent=67.9, used=11374669824, free=5382082560)


In [ ]:
print(len(df_Fund_Close_Period.index))

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

# No displays to protect PII data
#df_Fund_Close_Period.head(5)

In [ ]:
%who_ls DataFrame 

In [ ]:
del df_All_Switches
del df_All_Switches_Without_Fund_Close
del df_Closed_Funds_Policies_Removed
del df_Full_Population_Count
del df_Fund_Close_Period
del df_Fund_Close_Period_New
del df_Fund_Close_Pols
del df_Fund_Close_Unique_Pols
del df_Fund_NON_Close_Period
del df_LEFT_A
del df_Switches_Count

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')